In [22]:
import pandas as pd
import urllib.request
import csv
import matplotlib.pyplot as plt
import  platform
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup

In [2]:
# while True :
#     print("가수명-곡명 형태로 입력 : ",end='')
#     target_singer, target_title = input().split('-')
#     url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=' +urllib.parse.quote_plus(target_singer +' '+target_title + ' 곡정보')
#     html = urllib.request.urlopen(url).read()
#     soup = BeautifulSoup(html, 'html.parser')
    
#     try :
#         target_album = soup.select('.info.txt_4 > .info_group')[1].text.split()[1]
#         target_genre = soup.select('.info.txt_4 > .info_group')[3].text.strip().split()[1]
#         target_days = soup.select('.info.txt_4 > .info_group')[2].text.strip().split()[1].replace('.','')
#         season = soup.select('.info.txt_4 > .info_group')[2].text.strip().split('.')[1]
#         if '03' <= season and season <= '05':
#             target_season = '봄'
#         elif '06' <= season and season <= '08':
#             target_season = '여름'
#         elif '09' <= season and season <= '11':
#             target_season = '가을'
#         else:
#             target_season = '겨울'
#         target_lyric = soup.select('.text.no_ellipsis.type_center._content_text')[0].get_text()
#         break
        
#     # 곡이 없거나 잘못 입력된 경우    
#     except :
#         print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요")
#         pass
    
# print("장르 : " + target_genre + "\n" + "발매계절 : " + target_season)

# target_title_org = target_title.strip()
# target_singer_org = target_singer.strip()

# target_singer = target_singer.replace(' ','')
# target_title  = target_title.replace(' ','') 
# target_genre = target_genre.strip()
# target_season = target_season.strip()
# target_lyric = target_lyric.strip()
 

가수명-곡명 형태로 입력 : 조관우-겨울이야기
장르 : 발라드(국내)
발매계절 : 겨울


In [10]:
# if flag == 1 :
    
# while True:
#     data = pd.read_csv('가사전처리수정본.csv', low_memory=False)
#     if stop == 1 :
#         break
    
#     # 사용자가 입력한 곳이 데이터 셋에 있는지 확인
#     for i in range(len(data[["제목","가수"]])) :
#         temp_title = str(data.loc[i][0]).replace(' ','')
#         temp_singer = str(data.loc[i][1]).replace(' ','')
#         if (target_title in temp_title) and (target_singer in temp_singer) :
#             target_genre = data.loc[i][2]
#             target_season = data.loc[i][6]
#             stop = 1
#             break
    
#     # 없다면, 입력한 곡 추가
#     if stop == 0 :
#         temp = [target_title,target_singer,target_genre,target_days,target_album,target_lyric,target_season]
#         with open('가사전처리수정본.csv','a',encoding='utf-8-sig',newline='') as f:
#             Writer = csv.writer(f)
#             Writer.writerow(temp)
#         stop = 1
# end = time.time()
# print(end - start)

24.248034477233887


# data load
<br><br>
1. 약 42,000개의 크롤링 된 데이터 셋 csv 파일 읽어오기
<br><br>
2. 사용자의 input 부분 수정
    - '0' 입력시 종료
    - input 데이터 공백 제거
<br><br>
3. csv에서 입력된 곡 정보찾기
    - input data와 비교를 위해 data의 '제목','가사' replace를 이용하여 공백 제거
    - 곡이 있다면, index를 가져오는 부분 수정
    - 기존 : pd.Series(data.index, index=[data['제목'],data['가수']]).drop_duplicates()
    - 수정 후 : pd.Series(data.index, index=[data['제목'].replace(' ',''), data['가수'].replace(' ','')]).drop_duplicates()

<br><br>
4. 없다면 입력된 곡 가사 크롤링 후 기존 csv파일에 추가

In [23]:
import time
import pandas as pd
import urllib.request
import csv
import matplotlib.pyplot as plt
import  platform
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup

start = time.time()
data = pd.read_csv('가사전처리수정본2.csv', low_memory=False)
stop = flag = 0

while True:
    print("가수명-곡명 형태로 입력 (종료시 0 입력) : ", end='')
    temp_input = input()

    # 종료 조건
    if temp_input == '0':
        stop = 1
        break

    try:
        target_singer = temp_input.split('-')[0].replace(' ', '')
        target_title = temp_input.split('-')[1].replace(' ', '')

        if len(target_singer) == 0 or len(target_title) == 0:
            print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요.")
            continue

        df_and = data[(data['가수'].str.replace(' ','') == target_singer) & (data['제목'].str.replace(' ','') == target_title)]

        try:
            if len(df_and.index) > 0:
                print(1)
                # csv파일에 검색한 가수명과 곡명이 있는 경우

                indices = pd.Series(data.index, index=[data['제목'].replace(' ',''), data['가수'].replace(' ','')]).drop_duplicates()

                idx = indices[(target_title, target_singer)][0]

                target_genre = data['장르'].iloc[idx]
                target_season = data['계절'].iloc[idx]
                break

            else:
                print(2)
                # csv파일에 검색한 가수명과 곡명이 없는 경우

                url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=' + urllib.parse.quote_plus(
                    target_singer + ' ' + target_title + ' 곡정보')
                html = urllib.request.urlopen(url).read()
                soup = BeautifulSoup(html, 'html.parser')

                try:
                    # 입력한 곡 가사 크롤링
                    print(3)
                    title = soup.select_one('.area_text_title > ._text')
                    for i in title:
                        target_title = i
                    target_singer = soup.select('.info.txt_4 > .info_group')[0].text.strip().split()[1]
                    target_genre = soup.select('.info.txt_4 > .info_group')[3].text.strip().split()[1]
                    target_days = soup.select('.info.txt_4 > .info_group')[2].text.strip().split()[1].replace('.', '')
                    target_album = soup.select('.info.txt_4 > .info_group')[1].text.split()[1]
                    target_lyric = soup.select('.text.no_ellipsis.type_center._content_text')[0].get_text()
                    season = soup.select('.info.txt_4 > .info_group')[2].text.strip().split('.')[1]
                    if '03' <= season and season <= '05':
                        target_season = '봄'
                    elif '06' <= season and season <= '08':
                        target_season = '여름'
                    elif '09' <= season and season <= '11':
                        target_season = '가을'
                    else:
                        target_season = '겨울'
                    flag = 1
                    break

                except:
                    print(4)
                    # 곡이 없거나 잘못 입력된 경우
                    print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요.")
                    continue

        except:
            print(5)
            print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요.")
            continue

    # 곡이 없거나 잘못 입력된 경우
    except:
        print(6)
        print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요.")
        continue

# 곡을 추가해야 하는 경우
if flag == 1:
    target_genre = target_genre.strip()
    target_season = target_season.strip()
    temp = [target_title, target_singer, target_genre, target_days, target_album, target_lyric, target_season]
    with open('가사전처리수정본2.csv', 'a', encoding='utf-8-sig', newline='') as f:
        Writer = csv.writer(f)
        Writer.writerow(temp)
if stop == 0:
    print("가수: " + target_singer + "\n" + "제목: " + target_title + "\n" + "장르: " + target_genre + "\n" + "발매계절: " + target_season + "\n")
    print("실행시간 : " + str(time.time() - start))

가수명-곡명 형태로 입력 (종료시 0 입력) : 싸   이 - 어 땠   을 까
1


C:\Users\ILIFO-104\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2894: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


가수: 싸이
제목: 어땠을까
장르: 댄스(국내)
발매계절: 여름

실행시간 : 18.398060083389282


In [24]:
if data['가사'].isnull().sum() :
    data['가사'] = data['가사'].fillna('')
else :
    print("결측값 없음")

결측값 없음


## data 중복찾기
<br>
1.제목과 가수로 중복된 데이터 검색
<BR>
2.중복된 곡의 정보 조회

In [39]:
df = pd.DataFrame(data, columns=['제목','가수']) 
df2 = df[df.duplicated(['제목','가수'], keep = False)]
df_sorted_by_values = df2.sort_values(by='제목' ,ascending=False)
df_sorted_by_values

,제목,가수
8507,희야,이승철
16460,희야,이승철
16477,희야,이승철
11711,희망가,한대수
18718,희망가,한대수
...,...,...
4901,1994년 어느 늦은 밤,장혜진
11460,12월의 로망스,유리상자
14297,12월의 로망스,유리상자
14600,10월,모든


In [44]:
is_title_dup = data['제목'] == '희야'
title_dup = data[is_title_dup]
title_dup

,제목,가수,장르,발매일,앨범명,가사,계절
3342,희야,김태우,발라드,20150606,불후의 명곡 - 전설을 노래하다 (이승철편),\n너는 나를 좋아했잖아\n너는 비록 싫다고 말해도\n나는 너의 마음알아\nit's...,여름
5065,희야,부활,록메탈,20110620,Color,\n너는 비록 싫다고 말해도 나는 너의 마음 알아\n\n사랑한다 말하고 떠나면 나의...,여름
8507,희야,이승철,발라드,20051103,20th Anniversary Live In 2005,\n\n 나 좀 바라봐.\n너는 나를 좋아했잖아\n너는 비록 싫다고 말해도 나는 너...,가을
11237,희야,박일남,성인가요,20130805,언제 오시나요 / 그리운 소식,\n희야 오- 희야\n지금 어디 갔나\n나를 가슴 아픈 나를 버리고\n어이 떠나갔나...,여름
15273,희야,하예나,발라드,20120423,보이스 코리아 Part 3,\n너는 비록 싫다고 말해도 나는 너의 마음 알아\n\n사랑한다 말하고 떠나면 나의...,봄
16460,희야,이승철,록메탈,20080723,He`s Coming,\n너는 비록 싫다고 말해도 나는 너의 마음알아\n사랑한단 말하고 떠나면 나의 마음...,여름
16477,희야,이승철,발라드,20160628,The Gold (Original Recording Remastered),\n너는 나를 좋아했잖아 \n너는 비록 싫다고 말해도 \n나는 너의 마음 알아\n사...,여름


In [25]:
data.index 

RangeIndex(start=0, stop=19999, step=1)

In [26]:
#data = data.head(20000) 

In [27]:
# ngram_range = (1,2) 단어의 묶음을 1개부터 2개까지 설정
tfidf = TfidfVectorizer(stop_words='english',analyzer = 'word',  min_df=2, ngram_range = (2, 4), sublinear_tf=True)

# 가사에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['가사'])
print(tfidf_matrix.shape)

tfidf.fit(data['가사']) # 벡터라이저가 단어들을 학습합니다.
sorted(tfidf.vocabulary_.items()) # 단어사전을 정렬합니다. 
#tfidf.vocabulary_ # 벡터라이저가 학습한 단어사전을 출력합니다. 

(19999, 294731)


[('10 minute', 0),
 ('1000번의 이름을', 1),
 ('1000번의 이름을 불렀어', 2),
 ('1000번의 이름을 불렀어 1000개의', 3),
 ('1000일 동안', 4),
 ('1000일 동안 매일', 5),
 ('1000일 동안 매일 너의', 6),
 ('100프로 동의해', 7),
 ('100프로 동의해 너와의', 8),
 ('100프로 동의해 너와의 이별이', 9),
 ('100프로 한가한', 10),
 ('100프로 한가한 남자가', 11),
 ('100프로 한가한 남자가 돼줄게', 12),
 ('10cm씩 멀어져', 13),
 ('10cm씩 멀어져 가다', 14),
 ('10cm씩 멀어져 가다 가끔씩은', 15),
 ('10개월 매일', 16),
 ('10개월 매일 보는', 17),
 ('10개월 매일 보는 얼굴', 18),
 ('10년이 지나도', 19),
 ('10년이 지난', 20),
 ('10분 내로', 21),
 ('10분 내로 내게로', 22),
 ('10분 내로 내게로 달려와요', 23),
 ('10시 4분', 24),
 ('1234 bubble', 25),
 ('1234 bubble bubble', 26),
 ('1234 bubble bubble sweet', 27),
 ('12월에 그댈', 28),
 ('12월에 그댈 만났지', 29),
 ('12월의 아침을', 30),
 ('12월의 아침을 여는', 31),
 ('12월의 아침을 여는 베갯속', 32),
 ('18 18', 33),
 ('18 18 18', 34),
 ('1974년의 봄을', 35),
 ('1974년의 봄을 아직', 36),
 ('1974년의 봄을 아직 기억한다', 37),
 ('1994년 어느', 38),
 ('1994년 어느 늦은', 39),
 ('19살 섬색시가', 40),
 ('19살 섬색시가 순정을', 41),
 ('19살 섬색시가 순정을 바쳐', 42),
 ('1st verse', 43),
 ('1년도 버틸', 44),
 ('1년

## 코사인 유사도 실행

In [28]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 

## 유사곡 추천
1.사용자가 입력한 곡을 기준으로 장르, 발매계절이 같은 순으로 출력
<br>
2.장르가 같은것이 없다면, 발매계절이 같은 순으로 출력

In [29]:
def get_recommendations(title, singer, genre, season, cosine_sim=cosine_sim):
    # 선택한 음악의 가수, 제목으로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 음악를 가지고 연산할 수 있습니다.
    #idx = indices[title]
    idx = indices[(title, singer)][0]
    # 모든 음악에 대해서 해당 음악과의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 음악들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 1000개의 음악을 받아옵니다.
    sim_scores = sim_scores[1:1001]

    #코사인값은 두 벡터의 방향이 완전히 같을 경우 1, 90°의 각을 이룰 경우 0, 180°로 완전히 반대 방향인 경우 -1의 값을 갖음
    #print(sim_scores)
    
    # 가장 유사한 1000개의 음악의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 곡 출력
    num = 0
    for i in range(len(movie_indices)) :
        if num == 10 :
            break
        if float == type(data['장르'].iloc[movie_indices[i]]):
            continue
        if (data['장르'].iloc[movie_indices[i]] not in genre) :
            continue
        if(data['계절'].iloc[movie_indices[i]] != season) :
            continue
        num += 1
        a = data['가수'].iloc[movie_indices[i]] 
        b = data['제목'].iloc[movie_indices[i]]
        c = data['장르'].iloc[movie_indices[i]]
        d = data['계절'].iloc[movie_indices[i]]
        print(a+'-'+b+' ('+c+'), '+d+'발매') 
    
    if num < 10 :
        for i in range(len(movie_indices)) :
            if num == 10 :
                break
            if (data['계절'].iloc[movie_indices[i]] != season) :
                continue
            num += 1
            a = data['가수'].iloc[movie_indices[i]] 
            b = data['제목'].iloc[movie_indices[i]]
            c = data['장르'].iloc[movie_indices[i]]
            d = data['계절'].iloc[movie_indices[i]]
            print(a+'-'+b+' ('+c+'), '+d+'발매') 

In [30]:
get_recommendations(target_title, target_singer, target_genre, target_season)

C:\Users\ILIFO-104\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2940: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


자두-땡큐베뤼머취 (댄스), 여름발매
신혜성-Ex-Mind (댄스), 여름발매
손담비-dB Rider (댄스), 여름발매
유키스-Te Amo (댄스), 여름발매
김범수-외톨이야 (씨엔블루) (댄스), 여름발매
디유닛-기념일 (댄스), 여름발매
김현정-니가 사람이니! (댄스), 여름발매
히스토리-태양은 없다 (댄스), 여름발매
유키스-Believe (댄스), 여름발매
원투-둥글둥글 (댄스), 여름발매


## ngram_range = 2~4로 선정
<br>
아래 nrgam_range에 따른 결과를 보면, 1~2를 제외한 나머지의 결과는 동일
<br>
가사의 경우 1개의 단어보다 2개의 단어가 의미를 갖는 경우가 많다고 생각

1~2 : 19997,210240

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
바나나 보트-부비 Party (댄스), 가을발매
슈퍼걸-토요일 밤에 (댄스), 가을발매
투포케이-빨리와 (댄스), 가을발매
장우혁-Scream (댄스), 가을발매
원더보이즈-신난다 신나 (댄스), 가을발매
손호영-해피 바이러스 (댄스), 가을발매
탑독-비가 와서 그래 (댄스), 가을발매

1~4 : 19997,360352

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
엔소닉-Run & Run (댄스), 가을발매
거북이-사계 (댄스), 가을발매
빅스타-왜이래 (댄스), 가을발매
김완선, 용준형-Be Quiet (댄스), 가을발매
양파-뱀파이어 (댄스), 가을발매
유승준-Paradise (댄스), 가을발매
별-비밀하나 (댄스), 가을발매
인디고-그 누구보다 더 (댄스), 가을발매

1~5 : 19997,425464

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
엔소닉-Run & Run (댄스), 가을발매
거북이-사계 (댄스), 가을발매
빅스타-왜이래 (댄스), 가을발매
김완선, 용준형-Be Quiet (댄스), 가을발매
양파-뱀파이어 (댄스), 가을발매
유승준-Paradise (댄스), 가을발매
별-비밀하나 (댄스), 가을발매
인디고-그 누구보다 더 (댄스), 가을발매


2~4 : 19997,294707

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
엔소닉-Run & Run (댄스), 가을발매
거북이-사계 (댄스), 가을발매
빅스타-왜이래 (댄스), 가을발매
김완선, 용준형-Be Quiet (댄스), 가을발매
양파-뱀파이어 (댄스), 가을발매
유승준-Paradise (댄스), 가을발매
별-비밀하나 (댄스), 가을발매
인디고-그 누구보다 더 (댄스), 가을발매

2~5 : 19997,359819

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
엔소닉-Run & Run (댄스), 가을발매
거북이-사계 (댄스), 가을발매
빅스타-왜이래 (댄스), 가을발매
김완선, 용준형-Be Quiet (댄스), 가을발매
양파-뱀파이어 (댄스), 가을발매
유승준-Paradise (댄스), 가을발매
별-비밀하나 (댄스), 가을발매
인디고-그 누구보다 더 (댄스), 가을발매